# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myrandomforest consulting project').getOrCreate()
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [3]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [12]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], 
                            outputCol='features')
output = assembler.transform(data)
output = output.withColumn('label', output['Spoiled'])
output.show()

+---+---+----+---+-------+-------------------+-----+
|  A|  B|   C|  D|Spoiled|           features|label|
+---+---+----+---+-------+-------------------+-----+
|  4|  2|12.0|  3|    1.0| [4.0,2.0,12.0,3.0]|  1.0|
|  5|  6|12.0|  7|    1.0| [5.0,6.0,12.0,7.0]|  1.0|
|  6|  2|13.0|  6|    1.0| [6.0,2.0,13.0,6.0]|  1.0|
|  4|  2|12.0|  1|    1.0| [4.0,2.0,12.0,1.0]|  1.0|
|  4|  2|12.0|  3|    1.0| [4.0,2.0,12.0,3.0]|  1.0|
| 10|  3|13.0|  9|    1.0|[10.0,3.0,13.0,9.0]|  1.0|
|  8|  5|14.0|  5|    1.0| [8.0,5.0,14.0,5.0]|  1.0|
|  5|  8|12.0|  8|    1.0| [5.0,8.0,12.0,8.0]|  1.0|
|  6|  5|12.0|  9|    1.0| [6.0,5.0,12.0,9.0]|  1.0|
|  3|  3|12.0|  1|    1.0| [3.0,3.0,12.0,1.0]|  1.0|
|  9|  8|11.0|  3|    1.0| [9.0,8.0,11.0,3.0]|  1.0|
|  1| 10|12.0|  3|    1.0|[1.0,10.0,12.0,3.0]|  1.0|
|  1|  5|13.0| 10|    1.0|[1.0,5.0,13.0,10.0]|  1.0|
|  2| 10|12.0|  6|    1.0|[2.0,10.0,12.0,6.0]|  1.0|
|  1| 10|11.0|  4|    1.0|[1.0,10.0,11.0,4.0]|  1.0|
|  5|  3|12.0|  2|    1.0| [5.0,3.0,12.0,2.0]|

In [11]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
train_data, test_data = output.randomSplit([0.7, 0.3])
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)
dtc_preds.show()
rfc_preds.show()
gbt_preds.show()

+---+---+----+---+-------+-------------------+-----+-------------+-----------+----------+
|  A|  B|   C|  D|Spoiled|           features|label|rawPrediction|probability|prediction|
+---+---+----+---+-------+-------------------+-----+-------------+-----------+----------+
|  1|  1|12.0|  4|    1.0| [1.0,1.0,12.0,4.0]|  1.0|   [0.0,79.0]|  [0.0,1.0]|       1.0|
|  1|  1|13.0|  3|    1.0| [1.0,1.0,13.0,3.0]|  1.0|   [0.0,79.0]|  [0.0,1.0]|       1.0|
|  1|  3| 8.0|  3|    0.0|  [1.0,3.0,8.0,3.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  3| 8.0|  5|    0.0|  [1.0,3.0,8.0,5.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  4| 8.0|  5|    0.0|  [1.0,4.0,8.0,5.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  4| 8.0|  7|    0.0|  [1.0,4.0,8.0,7.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  5| 8.0|  5|    0.0|  [1.0,5.0,8.0,5.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  7| 7.0|  2|    0.0|  [1.0,7.0,7.0,2.0]|  0.0|  [231.0,0.0]|  [1.0,0.0]|       0.0|
|  1|  7| 

In [14]:
dtc_model.featureImportances

SparseVector(4, {0: 0.0049, 1: 0.0047, 2: 0.9576, 3: 0.0328})

In [16]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0221, 1: 0.023, 2: 0.9272, 3: 0.0277})

In [17]:
gbt_model.featureImportances

SparseVector(4, {0: 0.0047, 1: 0.023, 2: 0.88, 3: 0.0923})